In [ ]:
mat = [[1, 2, 3], 
    [4, 5, 6], 
    [7, 8, 19]]

In [1]:
import copy

In [3]:
class SingularMatrixException(Exception):
    def __init__(self):
        super().__init__('Matrix is Singular. Inverse does not exist')
        pass

In [4]:
class Sq_matrix:
    def __init__(self, mat: list):
        self.mat = mat
        self.n = len(mat)


    def gauss_eliminate(self):
        mat = self.mat
        
        mat1 = copy.deepcopy(mat)
        n = len(mat1)

        for j in range(0, n):
            pivot = mat1[j][j]

            if (pivot == 0):
                curr_max_indx = j
                curr_max_val = 0
                j2 = j+1
                for j2 in range(j+1, n): # move along rows
                    if abs(mat1[j2][j]) > curr_max_val:
                        curr_max_val = abs(mat1[j2][j])
                        curr_max_indx = j2

                if (j != curr_max_indx):
                    self.__swap_rows(j, curr_max_indx, mat1)
                    pivot = mat1[j][j]
                else:
                    continue

            for i in range(j+1, n):
                target = mat1[i][j]
                for k in range(j, n): 
                    mat1[i][k] = mat1[i][k] - (target/pivot)*mat1[j][k]
        
        return Sq_matrix(mat1)


    def rank(self):
        echelon_mat = self.gauss_eliminate()
        n = echelon_mat.n
        zero_rows = 0

        for i in range(n):
            zero_row_detected = True
            for j in range(n):
                if echelon_mat.mat[i][j] != 0:
                    zero_row_detected = False
                    break

            if zero_row_detected:
                zero_rows += 1
        rank = n - zero_rows
        return rank


    def det(self):
        upper_triang_mat = self.gauss_eliminate()
        mat = upper_triang_mat.mat

        det = 1
        for i in range(upper_triang_mat.n):
            det *= mat[i][i] # product of diagonal elements

        return det


    def inverse(self):
        n = self.n
        identity_mat = [[0 for i in range(n)] for j in range(n)]
        for i in range(n):
            identity_mat[i][i] = 1

        mat1 = copy.deepcopy(self.mat)

        # step1: reduce to upper triangular matrix
        for j in range(n):
            pivot = mat1[j][j]

            if (pivot == 0):
                curr_max_indx = j
                curr_max_val = 0
                j2 = j+1
                for j2 in range(j+1, n): # move along rows
                    if abs(mat1[j2][j]) > curr_max_val:
                        curr_max_val = abs(mat1[j2][j])
                        curr_max_indx = j2

                if (j != curr_max_indx):
                    self.__swap_rows(j, curr_max_indx, mat1, identity_mat)
                    pivot = mat1[j][j]
                else:
                    continue

            for i in range(j+1, n):
                target = mat1[i][j]
                for k in range(j, 2*n):
                    if k < n:
                        mat1[i][k] = mat1[i][k] - (target/pivot)*mat1[j][k]
                    else:
                        k2 = k - n
                        identity_mat[i][k2] = identity_mat[i][k2] - (target/pivot)*identity_mat[j][k2]


        # step2: reduce to diagonal matrix
        for j in range(n-1, -1, -1):
            pivot = mat1[j][j]

            if pivot == 0:
                raise SingularMatrixException()
                
            for i in range(j-1, -1, -1):
                target = mat1[i][j]
                for k in range(2*n-1, -1, -1):
                    if k >= n:
                        k2 = k - n
                        identity_mat[i][k2] = identity_mat[i][k2] - (target/pivot)*identity_mat[j][k2]
                    elif k > j:
                        continue
                    else:
                        mat1[i][k] = mat1[i][k] - (target/pivot)*mat1[j][k]
        
        
        # step3: reduce to identity matrix
        for i in range(n):
            pivot = mat1[i][i]
            mat1[i][i] = mat1[i][i] / pivot
            for j in range(n):
                # mat1[i][j] = mat1[i][j] / pivot
                identity_mat[i][j] = identity_mat[i][j] / pivot

        inverse_mat = identity_mat
        return Sq_matrix(inverse_mat)


    def transpose(self):
        n = self.n
        mat = copy.deepcopy(self.mat)

        for i in range(n):
            for j in range(i):
                mat[i][j], mat[j][i] = mat[j][i], mat[i][j]

        return Sq_matrix(mat)


    def largest_eigen_val(self):
        n = self.n

        eigen_vec = [1 for _ in range(n)]

        iter_count = 1000
        tolerance = 0.001
        eigen_val_prev = 0

        for iter_no in range(iter_count):
            eigen_vec = self.__mat_vec_mul(eigen_vec)

            curr_max = eigen_vec[0]
            for i in range(1, n):
                curr_max = max(curr_max, eigen_vec[i])

            for i in range(n):
                eigen_vec[i] /= curr_max

            eigen_val_new = curr_max

            if abs(eigen_val_new - eigen_val_prev) < tolerance:
                # print('Break', iter_no)
                break

            eigen_val_prev = eigen_val_new

        return eigen_val_new
    

    def __mat_vec_mul(self, vec):
        mat = self.mat
        n = self.n
        vec2 = [0 for _ in range(n)]

        for i in range(n):
            tmp = 0
            for j in range(n):
                tmp += mat[i][j] * vec[j]
            vec2[i] = tmp

        return vec2


    def __swap_rows(self, row1, row2, mat1, mat2 = None):
        n = len(mat[row1])
        for j in range(n):
            mat1[row1][j], mat1[row2][j] = mat1[row2][j], mat1[row1][j]
            if mat2:
                mat2[row1][j], mat2[row2][j] = mat2[row2][j], mat2[row1][j]
            

    @staticmethod
    def print_mat(mat):
        mat = mat.mat
        for row in mat:
            for elem in row:
                print(elem, end='\t')
            print()

In [5]:
mat = [[1,2,4], [1,2,5], [7,2,9]]

In [6]:
m = Sq_matrix(mat)

In [ ]:
m.gauss_eliminate()

In [7]:
Sq_matrix.print_mat(m.inverse())

0.666666666666667	-0.8333333333333335	0.16666666666666666	
2.1666666666666665	-1.5833333333333333	-0.08333333333333333	
-1.0	1.0	0.0	


In [ ]:
m.largest_eigen_val()

In [ ]:
Sq_matrix.print_mat(m)

In [ ]:
a = [[1, 2, 3, 4], 
[5, 6, 7, 8], 
[9, 0, 1, 2], 
[3, 4, 5, 6]]